In [2]:
import numpy as np
import cv2, os, random,shutil

Необходимо
заполнить параметры в следующей ячейке 
и запустить последовательно ячейки ниже

In [3]:
#На входе нам нужны:
#1. Структура видео:
#- Путь к видео файлу 1, имя класса 1, нужен ли поворот (применяется, если видео надо развернуть, если 0 то не нужен)
#- Путь к видео файлу 2, имя класса 2, нужен ли поворот (если 0 то не нужен)
#- Путь к видео файлу N, имя класса N, нужен ли поворот (если 0 то не нужен)
_VIDEO =  [
    ["D:\\GitHub\\Videos\\Jag.mp4","jag",0],
    ["D:\\GitHub\\Videos\\Brist.mp4","brist",0]
]

#2.Путь к папке в которой будут созданы файлы
OUT_FOLDER = "D:\\GitHub\\PhotoForDota2"

#3.На сколько процентов уменьшить фото (если не надо уменьшать поставить 100)
scale_percent = 50
#4.Надо ли проводить аугментацию
AUG = False
#5 Процент файлов Validation
_VALID = 20
#6 Процент файлов Test
_TEST = 10

# Процедуры и функции

In [4]:
# Основные процедуры преобразования
def start_video_to_frame():
    
    # создадим папки
    mk_dir_(os.path.join(OUT_FOLDER, "train"))
    mk_dir_(os.path.join(OUT_FOLDER, "validation"))
    mk_dir_(os.path.join(OUT_FOLDER, "test"))
    
    # создадим папки меток
    for i in _VIDEO:
        mk_dir_(os.path.join(OUT_FOLDER, "train/%s" %i[1] ))
        mk_dir_(os.path.join(OUT_FOLDER, "validation/%s" %i[1] ))
        mk_dir_(os.path.join(OUT_FOLDER, "test/%s" %i[1] ))
        
    # сначала все нарезаем в папку train
    index_video = 1
    for i in _VIDEO:
        # получили видео
        convert_video = i[0]
        # папка куда выгружаем
        dir_out = os.path.join(OUT_FOLDER, "train/%s" %i[1])
        # префикс складывеется из префика признака и порядкового номера
        prefix = i[1]+str(index_video)
        # признак необходимости перевернуть видео
        rotate = i[2]
        print("Start convert to ", convert_video,dir_out,prefix,rotate)
        # вызывыаем обработку видео
        video_to_frames(convert_video,dir_out,prefix,rotate)

    # а после того как все нарежем часть случайных файлов перенесем в остальные папки
    # получим пути к папкам
    dir_train = os.path.join(OUT_FOLDER, "train")
    dir_validation = os.path.join(OUT_FOLDER, "validation")
    dir_test = os.path.join(OUT_FOLDER, "test")
    # список папок в папке train
    dir_in_train = os.listdir(dir_train)
    for dir_prizn in dir_in_train:
        # путь к папке с признаками в папке train
        path_dir_prizn = os.path.join(dir_train, dir_prizn)
        # проврим, что это действительно папка 
        if os.path.isdir(path_dir_prizn):
            # полуим список файлов
            files = os.listdir(path_dir_prizn)
            # перемешаем их
            random.shuffle(files)
            # найдем количество файлов
            kol_files = len(files)
            # сколько пойдет в валидацию
            num_val = kol_files // 100 * _VALID
            # сколько пойдет в train
            num_test = kol_files // 100 * _TEST
            # массив со списком файлов для валидации
            files_val = files[0:num_val]
            # массив для списка файлов для теста
            files_test = files[num_val + 1:num_val + num_test]
            # обойдем все файлы из массива валидации
            for fv in files_val:
                # и скопируем файлы
                shutil.move(os.path.join(path_dir_prizn, fv),os.path.join(dir_validation, dir_prizn))

            # обойдем все файлы из массива test
            for fv in files_test:
                # и скопируем файлы
                shutil.move(os.path.join(path_dir_prizn, fv),os.path.join(dir_test, dir_prizn))
               
# функции нарезки фото из видео
def video_to_frames(video, dir_out, prefix,rotate):
    # extract frames from a video and save to directory as 'x.jpg' where 
    # x is the frame index
    #video = cv2.VideoCapture('/Users/alexeyvaganov/doc/files/foto_mp/pap.mov')
    vidcap = cv2.VideoCapture(video)
    count = 0
    while vidcap.isOpened():
        success, image = vidcap.read()
        if success:
            img_name = 'img_%s_%d.jpg' % (prefix,count)
            img_path = os.path.join(dir_out, img_name)
            image = autocrop(image, threshold=0)
            cv2.imwrite(img_path,image) 
            print("Start convert image ", img_path)
            #convert_image(img_path,rotate)
            if AUG:
                augmentation(dir_out,img_name)
            count += 1
        else:
            break
    # print(count)
    cv2.destroyAllWindows()
    vidcap.release()
    
# конвертировать картинку   
def convert_image(image_path,rotate): 
    
    image = cv2.imread(image_path)
    

    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    
    dim = (width, height)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    
    # и тут же перевренем
    if rotate != 0:
        image = rotate_bound(image, rotate)

    cv2.imwrite(image_path, image) 

# Повернуть картинку. параметры: картинка, угол
def rotate_bound(image, angle): 
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
    # perform the actual rotation and return the image

    return cv2.warpAffine(image, M, (nW, nH)) 

def autocrop(image, threshold=0):
    if len(image.shape) == 3:
        flatImage = np.max(image, 2)
    else:
        flatImage = image
    assert len(flatImage.shape) == 2

    rows = np.where(np.max(flatImage, 0) > threshold)[0]
    if rows.size:
        cols = np.where(np.max(flatImage, 1) > threshold)[0]
        image = image[cols[0]: cols[-1] + 1, rows[0]: rows[-1] + 1]
    else:
        image = image[:1, :1]
    
    return image

# создание папки
def mk_dir_ (path):
    if os.path.isdir(path):
        print ("Папка %s существует" %path )
    else:
        os.mkdir(path)
        print ("Создана папка %s " %path )

# функции аугментацию
def augmentation(dir_out,img_name):
    
    img_name_ws = img_name.replace(".jpg","_")
    
    img = cv2.imread(os.path.join(dir_out, img_name))

    
    img_aug = rotation(img,30)
    cv2.imwrite(os.path.join(dir_out,img_name_ws + "r+.jpg"),img_aug)
    
    img_aug = rotation(img,-30)
    cv2.imwrite(os.path.join(dir_out,img_name_ws + "r-.jpg"),img_aug)
    
    img_aug = vertical_flip(img, True)
    cv2.imwrite(os.path.join(dir_out,img_name_ws + "v.jpg"),img_aug)
    
    img_aug = horizontal_flip(img, True)
    cv2.imwrite(os.path.join(dir_out,img_name_ws + "t.jpg"),img_aug)
    
    img_aug = channel_shift(img, 50)
    cv2.imwrite(os.path.join(dir_out,img_name_ws + "c.jpg"),img_aug)
    
    img_aug = zoom(img, 0.2)
    cv2.imwrite(os.path.join(dir_out,img_name_ws + "z2.jpg"),img_aug)
    
    img_aug = zoom(img, 0.3)
    cv2.imwrite(os.path.join(dir_out,img_name_ws + "z3.jpg"),img_aug)
    
    img_aug = brightness(img, 0.5, 3)
    cv2.imwrite(os.path.join(dir_out,img_name_ws + "b.jpg"),img_aug)
    
    img_aug = vertical_shift(img, 0.7)
    cv2.imwrite(os.path.join(dir_out,img_name_ws + "vs.jpg"),img_aug)
    
    img_aug = horizontal_shift(img, ratio=0.7)
    cv2.imwrite(os.path.join(dir_out,img_name_ws + "gs.jpg"),img_aug)     

In [5]:
# вспомогательные функции для аугментации
# https://towardsdatascience.com/complete-image-augmentation-in-opencv-31a6b02694f5

# горизонтальное смещение
def horizontal_shift(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        print('Value should be less than 1 and greater than 0')
        return img
    ratio = random.uniform(-ratio, ratio)
    h, w = img.shape[:2]
    to_shift = w*ratio
    if ratio > 0:
        img = img[:, :int(w-to_shift), :]
    if ratio < 0:
        img = img[:, int(-1*to_shift):, :]
    img = fill(img, w, h)
    return img

# вертикальное смещение
def vertical_shift(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        print('Value should be less than 1 and greater than 0')
        return img
    ratio = random.uniform(-ratio, ratio)
    h, w = img.shape[:2]
    to_shift = h*ratio
    if ratio > 0:
        img = img[:int(h-to_shift), :, :]
    if ratio < 0:
        img = img[int(-1*to_shift):, :, :]
    img = fill(img, w, h)
    return img

# яркость
def brightness(img, low, high):
    value = random.uniform(low, high)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype = np.float64)
    hsv[:,:,1] = hsv[:,:,1]*value
    hsv[:,:,1][hsv[:,:,1]>255]  = 255
    hsv[:,:,2] = hsv[:,:,2]*value 
    hsv[:,:,2][hsv[:,:,2]>255]  = 255
    hsv = np.array(hsv, dtype = np.uint8)
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return img

# вспомогательная
def fill(img, h, w):
    img = cv2.resize(img, (h, w), cv2.INTER_CUBIC)
    return img

# зумиррование
def zoom(img, value):
    if value > 1 or value < 0:
        print('Value for zoom should be less than 1 and greater than 0')
        return img
    value = random.uniform(value, 1)
    h, w = img.shape[:2]
    h_taken = int(value*h)
    w_taken = int(value*w)
    h_start = random.randint(0, h-h_taken)
    w_start = random.randint(0, w-w_taken)
    img = img[h_start:h_start+h_taken, w_start:w_start+w_taken, :]
    img = fill(img, w, h)
    return img

# смещение канала
def channel_shift(img, value):
    value = int(random.uniform(-value, value))
    img = img + value
    img[:,:,:][img[:,:,:]>255]  = 255
    img[:,:,:][img[:,:,:]<0]  = 0
    img = img.astype(np.uint8)
    return img

# горизонтальное отображение
def horizontal_flip(img, flag):
    if flag:
        return cv2.flip(img, 1)
    else:
        return img

# вертикальное отображение
def vertical_flip(img, flag):
    if flag:
        return cv2.flip(img, 0)
    else:
        return img

#  сдвиг
def rotation(img, angle):
    angle = int(random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h))
    return img

# Запуск

In [6]:
start_video_to_frame()
print("Operation Complet")

Создана папка D:\GitHub\PhotoForDota2\train 
Создана папка D:\GitHub\PhotoForDota2\validation 
Создана папка D:\GitHub\PhotoForDota2\test 
Создана папка D:\GitHub\PhotoForDota2\train/jag 
Создана папка D:\GitHub\PhotoForDota2\validation/jag 
Создана папка D:\GitHub\PhotoForDota2\test/jag 
Создана папка D:\GitHub\PhotoForDota2\train/brist 
Создана папка D:\GitHub\PhotoForDota2\validation/brist 
Создана папка D:\GitHub\PhotoForDota2\test/brist 
Start convert to  D:\GitHub\Videos\Jag.mp4 D:\GitHub\PhotoForDota2\train/jag jag1 0
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_0.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_1.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_2.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_3.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_4.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_5.jpg
Start convert image  D:\GitHub\PhotoForDota2\trai

Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_108.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_109.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_110.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_111.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_112.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_113.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_114.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_115.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_116.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_117.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_118.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_119.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_120.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_

Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_225.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_226.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_227.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_228.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_229.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_230.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_231.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_232.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_233.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_234.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_235.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_236.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_237.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_

Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_340.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_341.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_342.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_343.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_344.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_345.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_346.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_347.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_348.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_349.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_350.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_351.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_352.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_

Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_457.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_458.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_459.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_460.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_461.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_462.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_463.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_464.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_465.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_466.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_467.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_468.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_469.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/jag\img_jag1_

...

Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_478.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_479.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_480.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_481.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_482.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_483.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_484.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_485.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_486.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_487.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_488.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_489.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_490.jpg
Start conver

Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_588.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_589.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_590.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_591.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_592.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_593.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_594.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_595.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_596.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_597.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_598.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_599.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_600.jpg
Start conver

Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_697.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_698.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_699.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_700.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_701.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_702.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_703.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_704.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_705.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_706.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_707.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_708.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_709.jpg
Start conver

Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_808.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_809.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_810.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_811.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_812.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_813.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_814.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_815.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_816.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_817.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_818.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_819.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_820.jpg
Start conver

Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_919.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_920.jpg
Start convert image  D:\GitHub\PhotoForDota2\train/brist\img_brist1_921.jpg
Operation Complet


# Test